In [32]:
import pandas as pd
import numpy as np
import random

from sklearn.model_selection import KFold,StratifiedKFold

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, Normalizer, PowerTransformer, RobustScaler
from sklearn.decomposition import PCA

from sklearn.ensemble import GradientBoostingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn.neighbors import KNeighborsRegressor
import xgboost
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

from keras import Input
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import (mean_squared_error,
                             r2_score,
                             mean_absolute_percentage_error,
                             mean_absolute_error
                            )

from sklearn.pipeline import Pipeline
from statsmodels.tsa.api import VAR

from sklearn.feature_selection import SelectKBest,f_regression

In [33]:
df = pd.read_csv('../../data/dataset/Full_data_File.csv')

In [34]:
df.head()

,location,country,latitude,longitude,generationtime_ms,elevation,Date,year,month,day,...,RH,Dew_Point,snowfall,Pressure,wind_speed,wind_direction,wind_gusts,sunshine_duration,direct_radiation,AQI
0,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-01,2021,1,1,...,92.208333,1.266667,0.0,1006.029167,7.208333,243.250000,11.187500,890.836250,16.208333,65.125000
1,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-02,2021,1,2,...,96.708333,1.312500,0.0,1011.245833,6.445833,216.916667,10.283333,937.792917,22.625000,33.291667
2,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-03,2021,1,3,...,91.458333,1.229167,0.0,1015.504167,14.333333,52.208333,25.516667,739.805000,17.791667,34.041667
3,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-04,2021,1,4,...,86.291667,0.879167,0.0,1017.016667,25.041667,49.375000,45.595833,0.000000,1.041667,19.750000
4,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-05,2021,1,5,...,91.125000,1.475000,0.0,1016.929167,18.708333,54.541667,34.145833,46.027083,1.833333,10.166667


In [35]:
df.shape

(49674, 27)

In [36]:
def get_important_Feature(df):
    
    def get_train_test(df):
        X = df.iloc[:,0:-1]
        y = df.iloc[:,-1]

        X = MinMaxScaler().fit_transform(X)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        return X_train, X_test, y_train, y_test
    
    def fitness_function(selected_features, X_train, X_test, y_train, y_test):
        # Select features based on the particle's position
        selected_features = np.array(selected_features, dtype=bool)
        if not any(selected_features):  # Avoid empty feature selection
            return float('inf')  # Assign a very high MSE

        X_train_selected = X_train[:, selected_features]
        X_test_selected = X_test[:, selected_features]

        # Train a regression model
        model = DecisionTreeRegressor(random_state=42)
        model.fit(X_train_selected, y_train)
        predictions = model.predict(X_test_selected)

        # Calculate and return MSE
        mse = mean_squared_error(y_test, predictions)
        return mse
    
    def pso_feature_selection(n_particles, n_iterations, n_features, df):
        # Initialize particles
        particles = [np.random.choice([0, 1], size=n_features) for _ in range(n_particles)]
        velocities = [np.random.uniform(-1, 1, size=n_features) for _ in range(n_particles)]
        personal_best_positions = particles.copy()
        personal_best_scores = [float('inf')] * n_particles
        global_best_position = None
        global_best_score = float('inf')

        X_train, X_test, y_train, y_test = get_train_test(df)

        # PSO loop
        for iteration in range(n_iterations):
            for i, particle in enumerate(particles):
                # Evaluate fitness
                mse = fitness_function(particle, X_train, X_test, y_train, y_test)

                # Update personal best
                if mse < personal_best_scores[i]:
                    personal_best_scores[i] = mse
                    personal_best_positions[i] = particle.copy()

                # Update global best
                if mse < global_best_score:
                    global_best_score = mse
                    global_best_position = particle.copy()

            # Update particle velocities and positions
            for i, particle in enumerate(particles):
                r1, r2 = np.random.rand(n_features), np.random.rand(n_features)
                cognitive = r1 * (personal_best_positions[i] - particle)
                social = r2 * (global_best_position - particle)
                velocities[i] = 0.5 * velocities[i] + 1.5 * cognitive + 1.5 * social
                particles[i] = np.where(np.random.rand(n_features) < 1 / (1 + np.exp(-velocities[i])), 1, 0)  # Sigmoid activation

            # Print progress
#             print(f"Iteration {iteration+1}/{n_iterations}, Best MSE: {global_best_score}")

        return global_best_position, global_best_score



    n_particles = 20
    n_iterations = 50
    n_features = len(list(df)) - 1

    # Run PSO
    best_features, best_score = pso_feature_selection(n_particles, n_iterations, n_features, df)

    best_feature_list = []
    for name_, flag in zip(list(df)[:-1],best_features):
        if flag == 1:
            best_feature_list.append(name_)
    best_feature_list.append(list(df)[-1])
    return df[best_feature_list], best_feature_list

In [37]:
# def get_important_Feature(df):
#     X = df.values[:,0:-1]
#     y = df.values[:,-1]
#     fs = SelectKBest(score_func=f_regression,k='all')
#     fs.fit(X,y)
#     score_df = pd.DataFrame([fs.pvalues_,fs.scores_]).T
#     score_df.columns = ['p-value','score']
#     score_df['columns'] = list(df)[:-1]
#     score_df = score_df[score_df['p-value'] < 0.05].copy()
    
#     feature_col = score_df['columns'].to_list()
#     target_col = [list(df)[-1]]
    
#     return df[feature_col + target_col].copy(), feature_col

In [38]:
def get_k_fold_operation_data_proposed_approach(df, date_, cv):
    
    df = pd.DataFrame(df)
    colLen = len(list(df))

    X = df.values[:, 0:colLen - 1]
    y = df.values[:, colLen - 1]
    
    
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    date_train_list = []
    date_test_list = []
    
    kf = KFold(n_splits=cv)
    
    for train_index,test_index in kf.split(X,y):
        X_train, X_test, y_train, y_test, date_train, date_test  = X[train_index], X[test_index], y[train_index], y[test_index], date_[train_index], date_[test_index]
        
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
        
        date_train_list.append(date_train)
        date_test_list.append(date_test)
        
    
        
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list

In [39]:
def get_IQR_Operation_value(df, col, cv):
    
    S_B = df.shape[0]
    
    Q1 = df['AQI'].quantile(0.25)
    Q3 = df['AQI'].quantile(0.75)

    # Calculate IQR
    IQR = Q3 - Q1

    lower_bound = Q1 - 1.0 * IQR
    upper_bound = Q3 + 1.75 * IQR
    
    print(lower_bound,upper_bound)
    
#     df = df.loc[(df['AQI'] <= upper_bound) & (df['AQI'] >= lower_bound)]
    date_ = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day)).values
    df, impt_col = get_important_Feature(df)

    X_train_list, \
    X_test_list, \
    y_train_list, \
    y_test_list, \
    date_train_list, \
    date_test_list = get_k_fold_operation_data_proposed_approach(df, date_, cv)
    
    S_A = df.shape[0]
    
    diff = S_B - S_A
    
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list, diff, impt_col
    

In [40]:
def get_k_fold_operation_data(df, cv):
    
    df = pd.DataFrame(df)
    colLen = len(list(df))

    X = df.values[:, 0:colLen - 1]
    y = df.values[:, colLen - 1]
    date_ = pd.to_datetime(dict(year=df.year, month=df.month, day=df.day)).values
    
    
    X_train_list = []
    X_test_list = []
    y_train_list = []
    y_test_list = []
    date_train_list = []
    date_test_list = []
    
    kf = KFold(n_splits=cv)
    
    for train_index,test_index in kf.split(X,y):
        X_train, X_test, y_train, y_test, date_train, date_test  = X[train_index], X[test_index], y[train_index], y[test_index], date_[train_index], date_[test_index]
        
        X_train_list.append(X_train)
        X_test_list.append(X_test)
        y_train_list.append(y_train)
        y_test_list.append(y_test)
        
        date_train_list.append(date_train)
        date_test_list.append(date_test)
        
    
        
    return X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list

In [41]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, mean_absolute_percentage_error
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler, PowerTransformer, RobustScaler
import tensorflow as tf


def NN_Regression(X_train, X_test, y_train, y_test, scaler_name):

    # Initialize the scaler
    scaler = None
    if scaler_name == 'StandardScaler':
        scaler = StandardScaler()
    elif scaler_name == 'Normalizer':
        scaler = Normalizer()
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
    elif scaler_name == 'PowerTransformer':
        scaler = PowerTransformer(method='yeo-johnson')
    elif scaler_name == 'RobustScaler':
        scaler = RobustScaler()

    # Apply scaling
    if scaler_name == 'not_processing':
        X_train_scaled = X_train
        X_test_scaled = X_test
    else:
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)

    random.seed(1)
    np.random.seed(1)
    tf.keras.utils.set_random_seed(1)

    # Build the Neural Network model
    def build_nn(input_dim):
        model = Sequential([
            Input(shape=(input_dim,)),  # Define input shape
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(1, activation='linear')  # Regression output layer
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')  # Compile the model
        return model

    nn = build_nn(X_train_scaled.shape[1])

    # Train the Neural Network
    nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # Predict using the Neural Network
    y_pred = nn.predict(X_test_scaled).flatten()

    # Evaluate the Neural Network
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100

    return mae, rmse, r_2, mape, y_pred


In [42]:
def NN_GBR(X_train, X_test, y_train, y_test, scaler_name, model_object):
    scaler = None
    # Step 1: Initialize the scaler
    if scaler_name == 'StandardScaler':
        scaler = StandardScaler()
        
    elif scaler_name == 'Normalizer':
        scaler = Normalizer()
        
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
    
    elif scaler_name == 'PowerTransformer':
        scaler = PowerTransformer(method='yeo-johnson')
        
    elif scaler_name == 'RobustScaler':
        scaler = RobustScaler()
    
    ###############################################
    if scaler_name == 'not_processing':
        X_train_scaled = X_train
        X_test_scaled = X_test
    else:
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
    ###############################################
    
    
    X_train_pca = X_train_scaled
    X_test_pca = X_test_scaled
    
    import tensorflow as tf
    tf.keras.utils.set_random_seed(1)
    
    # Step 3: Build a Neural Network for feature extraction
    def build_nn(input_dim):
        model = Sequential([
            Input(shape=(input_dim,)),  # Use Input layer for the shape
            Dense(64, activation='relu'),
            Dense(32, activation='relu'),
            Dense(1, activation='linear')
        ])
        model.compile(optimizer=Adam(learning_rate=0.001), loss='mse')
        return model

    nn = build_nn(X_train_pca.shape[1])
    nn.fit(X_train_pca, y_train, epochs=50, batch_size=32, validation_split=0.2, verbose=0)

    # Step 4: Extract features from the Neural Network
    # Extract the second-to-last layer's output
    extractor = Sequential(nn.layers[:-1])
    X_train_nn_features = extractor.predict(X_train_pca)
    X_test_nn_features = extractor.predict(X_test_pca)

    # Step 5: Train a Gradient Boosting Regressor
    gbr = model_object
    gbr.fit(X_train_nn_features, y_train)

    # Step 6: Evaluate the model
    y_pred = gbr.predict(X_test_nn_features)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    return mae, rmse, r_2, mape, y_pred

In [43]:
def GBR(X_train, X_test, y_train, y_test, scaler_name, model_object):
    scaler = None
    # Step 1: Initialize the scaler
    if scaler_name == 'StandardScaler':
        scaler = StandardScaler()
        
    elif scaler_name == 'Normalizer':
        scaler = Normalizer()
        
    elif scaler_name == 'MinMaxScaler':
        scaler = MinMaxScaler()
        
    elif scaler_name == 'PowerTransformer':
        scaler = PowerTransformer(method='yeo-johnson')
        
    elif scaler_name == 'RobustScaler':
        scaler = RobustScaler()
    
    ###############################################
    if scaler_name == 'not_processing':
        pipeline = Pipeline([
            ('gbr', model_object)
        ])
    else:
        pipeline = Pipeline([
            ('scaler', scaler),
            ('gbr', model_object)
        ])
    ###############################################

    # Fit pipeline
    pipeline.fit(X_train, y_train)
    y_pred = pipeline.predict(X_test)
    
    mae = mean_absolute_error(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    r_2 = r2_score(y_test, y_pred)
    mape = mean_absolute_percentage_error(y_test, y_pred) * 100
    
    return mae, rmse, r_2, mape, y_pred

In [44]:
model_dict = {
#     'GradientBoostingRegressor':GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=3, random_state=42),
#     'RandomForestRegressor':RandomForestRegressor(random_state=42,n_estimators=300),
#     'AdaBoostRegressor': AdaBoostRegressor(n_estimators=100,random_state=42),
#     'xgboost.XGBRegressor' : xgboost.XGBRegressor(objective='reg:linear',n_estimators = 100, random_state=42)
#     'KNeighborsRegressor' : KNeighborsRegressor(n_neighbors=5),
    'LinearRegression' : LinearRegression()
#     'DecisionTreeRegressor' : DecisionTreeRegressor(random_state=42)
}

In [45]:
cv  =5
summary_value = []
detail_value = []
meta_data = []

for city_ in df['location'].unique():
    city_df = df[df['location'] == city_]
    country_ = city_df['country'].unique()[0]
    
    print(city_)
    city_df.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)
    
    target_col = list(city_df)[-1]
    X_train_list, \
    X_test_list, \
    y_train_list, \
    y_test_list, \
    date_train_list, \
    date_test_list, \
    diff, \
    impt_col = get_IQR_Operation_value(city_df,target_col,cv)
    
    meta_data.append(country_)
    meta_data.append(city_)
    meta_data.append(diff)
    meta_data.append(city_df.shape[0])
    meta_data.append(city_df.shape[1])
    meta_data.append(impt_col)
    
    
    k_fold = 1
    
    mae_list = []
    rmse_list = []
    r_2_list = []
    mape_list = []
    
    for X_train, X_test, y_train, y_test, date_train, date_test in zip(X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list):
        for _preprocessing_name in ['StandardScaler']: #'RobustScaler', 'MinMaxScaler' , 'StandardScaler'
            for model_name, _ in model_dict.items():

                model_object = model_dict[model_name]

                res = []
                mae, rmse, r_2, mape, y_pred = NN_GBR(X_train, X_test, y_train, y_test, _preprocessing_name, model_object)

                res.append(country_)
                res.append(city_)
                res.append(mae)
                res.append(rmse)
                res.append(r_2)
                res.append(mape)
                res.append('fold_' + str(k_fold))
                res.append(_preprocessing_name)
                res.append('NN-ML')
                res.append(model_name)
                print(res)

                full_output = pd.DataFrame({
                    'actual_value' : y_test,
                    'predict_value' : y_pred,
                    'Fold_number' : ['fold_' + str(k_fold)] * len(y_test),
                    'date' : date_test
                })
                full_output['city'] = city_
                full_output['country'] = country_
                full_output['pre-processing name'] = _preprocessing_name
                full_output['method'] = 'NN-ML'
                full_output['model_name'] = model_name

                summary_value.append(res)
                detail_value.append(full_output)

        k_fold = k_fold + 1
        
        

Amsterdam
13.541666666666668 41.66666666666667


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\3734773123.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Netherlands', 'Amsterdam', 1.1359901656062936, 1.4244278358127362, 0.970463166910633, 4.617564238863571, 'fold_1', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Netherlands', 'Amsterdam', 1.0892540308982817, 1.3982263778390023, 0.9804032123674619, 4.593044616335813, 'fold_2', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
['Netherlands', 'Amsterdam', 0.9202199548346812, 1.1258832806438428, 0.9840097272153175, 3.848706182663147, 'fold_3', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Netherlands', 'Amsterdam', 0.9765443649466177, 1.2649587794098236, 0.9694590226115661, 4.3388783052079445, 'fold_4', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━

C:\Users\HP\AppData\Local\Temp\ipykernel_20712\3734773123.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Greece', 'Athens', 1.036034528712771, 1.5332725888523087, 0.9777558082703109, 2.592216738008245, 'fold_1', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Greece', 'Athens', 0.9921871320297726, 1.312776704690273, 0.9779379920497614, 3.0758954205246636, 'fold_2', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
['Greece', 'Athens', 0.9984051038141122, 1.2676484339874794, 0.9834573363070799, 3.0221231391333423, 'fold_3', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Greece', 'Athens', 1.3444190482570704, 1.759424435848204, 0.9563311624309221, 4.026626379991036, 'fold_4', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━

...

37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Sewden', 'Stockholm', 1.0081469066042672, 1.3269021162713235, 0.9458926587529702, 4.816925783750891, 'fold_1', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Sewden', 'Stockholm', 0.9130804462519954, 1.3458239589574275, 0.950677515016639, 4.00123509726844, 'fold_2', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Sewden', 'Stockholm', 0.8482785899889522, 1.1561097144819001, 0.9652866982681586, 4.181572196000646, 'fold_3', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Sewden', 'Stockholm', 0.9675611611370626, 1.1940350857634248, 0.9296206395422062, 4.770677989797479, 'fold_4', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10

C:\Users\HP\AppData\Local\Temp\ipykernel_20712\3734773123.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step 
['Switzerland', 'bern', 0.4152798744869992, 0.6068102585252803, 0.9926643198261368, 1.68579379922675, 'fold_1', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Switzerland', 'bern', 0.4374415297486467, 0.5993715730470878, 0.9951743163008825, 1.6993387758884548, 'fold_2', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Switzerland', 'bern', 0.4056580404168394, 0.5464481288577752, 0.9955653572580334, 1.8790132659410022, 'fold_3', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Switzerland', 'bern', 0.5037098573223098, 0.7978438324245919, 0.9890407019156701, 2.24062567572848, 'fold_4', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\3734773123.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location', 'country', 'latitude', 'longitude', 'generationtime_ms','elevation','Date'],inplace=True)


37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Luxembourg', 'luxembourg findel airport', 1.1660764767990728, 1.4850179311538085, 0.9596374600526044, 5.110543363433004, 'fold_1', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Luxembourg', 'luxembourg findel airport', 0.8309345876789528, 1.0901126923190687, 0.9809720139318101, 3.4286121344304816, 'fold_2', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Luxembourg', 'luxembourg findel airport', 0.9395914273719259, 1.2165733711414393, 0.9776518488042779, 4.159722352618005, 'fold_3', 'StandardScaler', 'NN-ML', 'LinearRegression']
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step 
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
['Luxembourg', 'luxembourg findel airport', 0.9451622113789596, 1.258142139903999, 0.9700018809941688, 4.184936442001123, 'fold_4', 'Standa

In [46]:
summary_df = pd.DataFrame(summary_value,columns=['Country','City','MAE','RMSE','R2','MAPE','K_Fold','pre-processing name','method','model_name'])

In [47]:
detail_df = pd.concat(detail_value)

In [48]:
summary_df.to_csv('../_output/AQI_Proposed_Approach_Summary_result.csv',index=False)

In [49]:
detail_df.to_csv('../_output/AQI_Proposed_Approach_Detail_result.csv',index=False)

In [50]:
meta_df = pd.DataFrame(np.reshape(np.array(meta_data, dtype="object"),(int(len(meta_data)/6),6)),columns=['Country','City',
                                                                                               'IQR_Value','total_row',
                                                                                               'total_column','important_feature'])

In [51]:
meta_df.to_csv('../_output/AQI_Proposed_Approach_metadata.csv',index=False)

In [52]:
detail_df['method'].unique()

array(['NN-ML'], dtype=object)

In [53]:
model_dict = {
#     'GradientBoostingRegressor':GradientBoostingRegressor(n_estimators=200, learning_rate=0.05, max_depth=3, random_state=42),
    'RandomForestRegressor':RandomForestRegressor(random_state=42,n_estimators=300),
#     'AdaBoostRegressor': AdaBoostRegressor(n_estimators=100,random_state=42),
#     'xgboost.XGBRegressor' : xgboost.XGBRegressor(objective='reg:linear',n_estimators = 100, random_state=42),
    'KNeighborsRegressor' : KNeighborsRegressor(n_neighbors=5),
    'LinearRegression' : LinearRegression(),
    'DecisionTreeRegressor' : DecisionTreeRegressor(random_state=42)
}

In [54]:
df.head(1)

,location,country,latitude,longitude,generationtime_ms,elevation,Date,year,month,day,...,RH,Dew_Point,snowfall,Pressure,wind_speed,wind_direction,wind_gusts,sunshine_duration,direct_radiation,AQI
0,Amsterdam,Netherlands,52.4,4.9,104.943037,14,2021-01-01,2021,1,1,...,92.208333,1.266667,0.0,1006.029167,7.208333,243.25,11.1875,890.83625,16.208333,65.125


In [55]:
cv = 5
summary_value = []
detail_value = []

for city_ in df['location'].unique():
    city_df = df[df['location'] == city_]
    country_ = city_df['country'].unique()[0]
    
    print(city_)
    city_df.drop(columns=['location', 
                          'country', 
                          'latitude', 
                          'longitude', 
                          'generationtime_ms',
                          'elevation',
                          'Date'
                         ],inplace=True)
    
    target_col = list(city_df)[-1]
    
    X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list = get_k_fold_operation_data(city_df, cv)
    
    k_fold = 1
    
    mae_list = []
    rmse_list = []
    r_2_list = []
    mape_list = []
    
    for X_train, X_test, y_train, y_test, date_train, date_test in zip(X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list):
        for _preprocessing_name in ['StandardScaler']: #'RobustScaler', 'MinMaxScaler' , 'StandardScaler'
            for method_name in ['ML']: # 
                for model_name, _ in model_dict.items():
                    
                    model_object = model_dict[model_name]
                
                    res = []
                    mae, rmse, r_2, mape, y_pred = GBR(X_train, X_test, y_train, y_test, _preprocessing_name, model_object)
                    
                    res.append(country_)
                    res.append(city_)
                    res.append(mae)
                    res.append(rmse)
                    res.append(r_2)
                    res.append(mape)
                    res.append('fold_' + str(k_fold))
                    res.append(_preprocessing_name)
                    res.append(method_name)
                    res.append(model_name)
                    print(res)

                    full_output = pd.DataFrame({
                        'actual_value' : y_test,
                        'predict_value' : y_pred,
                        'Fold_number' : ['fold_' + str(k_fold)] * len(y_test),
                        'date' : date_test
                    })
                    full_output['city'] = city_
                    full_output['country'] = country_
                    full_output['pre-processing name'] = _preprocessing_name
                    full_output['method'] = method_name
                    full_output['model_name'] = model_name

                    summary_value.append(res)
                    detail_value.append(full_output)

        k_fold = k_fold + 1
        
        

Amsterdam


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\2764872612.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location',


['Netherlands', 'Amsterdam', 0.7617647895335631, 1.0879977286870095, 0.9827678684101576, 3.263698220397561, 'fold_1', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Netherlands', 'Amsterdam', 2.5993458475540385, 3.4096510494075236, 0.8307600476821946, 10.143414510574567, 'fold_1', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Netherlands', 'Amsterdam', 2.31795066963966, 2.902884021113702, 0.8773288618921631, 8.954984270450911, 'fold_1', 'StandardScaler', 'ML', 'LinearRegression']
['Netherlands', 'Amsterdam', 1.0878839590443687, 1.6011659692642957, 0.9626787890061967, 4.5904089886939845, 'fold_1', 'StandardScaler', 'ML', 'DecisionTreeRegressor']
['Netherlands', 'Amsterdam', 0.7399923896499243, 1.0890858496136657, 0.9881107588741659, 3.1110451180080942, 'fold_2', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Netherlands', 'Amsterdam', 2.438784246575342, 3.301756867387334, 0.8907251839634848, 9.207008939275827, 'fold_2', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Netherl

C:\Users\HP\AppData\Local\Temp\ipykernel_20712\2764872612.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location',


['Greece', 'Athens', 1.3789723170269264, 2.2436498460155825, 0.9523691813055851, 3.3938151647358112, 'fold_1', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Greece', 'Athens', 3.6675199089874857, 4.688694052191397, 0.7919912153905061, 9.12758977878819, 'fold_1', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Greece', 'Athens', 3.47995412811465, 4.525823747354683, 0.8061913482910916, 8.890864577189092, 'fold_1', 'StandardScaler', 'ML', 'LinearRegression']
['Greece', 'Athens', 1.6572810011376562, 2.3628375984978813, 0.9471742547900404, 4.281259386298084, 'fold_1', 'StandardScaler', 'ML', 'DecisionTreeRegressor']
['Greece', 'Athens', 0.8402216514459677, 1.2187452026513614, 0.9809853142615715, 2.6226994817229494, 'fold_2', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Greece', 'Athens', 3.131021689497717, 4.140431934917714, 0.7805403850257303, 9.107808004432414, 'fold_2', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Greece', 'Athens', 2.301449958010385, 2.8212398484780796, 

...

['Sewden', 'Stockholm', 0.5213035646568053, 0.9893671529034207, 0.9699189220071951, 2.2988423474079482, 'fold_1', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Sewden', 'Stockholm', 2.2446245733788395, 2.91010618421613, 0.7397467916597154, 10.295343754249833, 'fold_1', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Sewden', 'Stockholm', 1.3094417842970634, 1.767446544863101, 0.9040000524222824, 5.76925620231451, 'fold_1', 'StandardScaler', 'ML', 'LinearRegression']
['Sewden', 'Stockholm', 0.6860068259385668, 1.416060065539247, 0.9383771512806782, 2.975037950814416, 'fold_1', 'StandardScaler', 'ML', 'DecisionTreeRegressor']
['Sewden', 'Stockholm', 0.7369140030441377, 2.813538838717664, 0.7844369759886947, 2.572023797841097, 'fold_2', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Sewden', 'Stockholm', 2.2175799086757984, 3.69366371455568, 0.6284791905435133, 8.738034855664683, 'fold_2', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Sewden', 'Stockholm', 1.5223199316506653,

C:\Users\HP\AppData\Local\Temp\ipykernel_20712\2764872612.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location',


['Switzerland', 'bern', 0.4774454872961696, 0.836494886604886, 0.9860600524080333, 1.7822508302601459, 'fold_1', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Switzerland', 'bern', 2.7244880546075088, 3.410892516734137, 0.7682228446098724, 10.550210823996391, 'fold_1', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Switzerland', 'bern', 2.412684064491519, 3.4355495660352484, 0.7648597382514594, 8.79542335780709, 'fold_1', 'StandardScaler', 'ML', 'LinearRegression']
['Switzerland', 'bern', 0.6113481228668945, 1.0053746115723412, 0.9798632031912465, 2.354489320121369, 'fold_1', 'StandardScaler', 'ML', 'DecisionTreeRegressor']
['Switzerland', 'bern', 0.43948439878234324, 0.7755775222255764, 0.9919198951292012, 1.6157869490791639, 'fold_2', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Switzerland', 'bern', 2.6405821917808208, 3.6020500489019462, 0.8257125742183561, 9.688769605625373, 'fold_2', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Switzerland', 'bern', 2.61206058708

C:\Users\HP\AppData\Local\Temp\ipykernel_20712\2764872612.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location',


['Luxembourg', 'luxembourg findel airport', 0.4461566173682207, 0.7251162219759576, 0.9903765601826975, 1.890471224166897, 'fold_1', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Luxembourg', 'luxembourg findel airport', 2.304436860068259, 2.9061356281090416, 0.8454222967827254, 9.759172867163628, 'fold_1', 'StandardScaler', 'ML', 'KNeighborsRegressor']
['Luxembourg', 'luxembourg findel airport', 2.231078527417601, 3.2085503425599935, 0.8115774798601969, 8.61139595537255, 'fold_1', 'StandardScaler', 'ML', 'LinearRegression']
['Luxembourg', 'luxembourg findel airport', 0.6217292377701935, 1.093020268242093, 0.9781338902733762, 2.588349221192391, 'fold_1', 'StandardScaler', 'ML', 'DecisionTreeRegressor']
['Luxembourg', 'luxembourg findel airport', 0.4268902207001501, 0.6778506151874606, 0.9926427075169914, 1.7776247092551334, 'fold_2', 'StandardScaler', 'ML', 'RandomForestRegressor']
['Luxembourg', 'luxembourg findel airport', 2.810416666666667, 4.175311893079379, 0.7208562699647845

In [56]:
summary_df = pd.DataFrame(summary_value,columns=['Country','City','MAE','RMSE','R2','MAPE','K_Fold','pre-processing name','method','model_name'])
detail_df = pd.concat(detail_value)
summary_df.to_csv('../_output/AQI_ML_Summary_result.csv',index=False)
detail_df.to_csv('../_output/AQI_ML_Detail_result.csv',index=False)

In [57]:
summary_value = []
detail_value = []

for city_ in df['location'].unique():
    city_df = df[df['location'] == city_]
    country_ = city_df['country'].unique()[0]
    
    print(city_)
    city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)
    
    target_col = list(city_df)[-1]
#     city_df = get_IQR_Operation_value(city_df,target_col)
    
    X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list = get_k_fold_operation_data(city_df, 5)
    
    k_fold = 1
    
    mae_list = []
    rmse_list = []
    r_2_list = []
    mape_list = []
    
    for X_train, X_test, y_train, y_test, date_train, date_test in zip(X_train_list, X_test_list, y_train_list, y_test_list, date_train_list, date_test_list):
        for _preprocessing_name in ['StandardScaler']: #'RobustScaler', 'MinMaxScaler' , 'StandardScaler'
            
            res = []
            
            mae, rmse, r_2, mape, y_pred = NN_Regression(X_train, X_test, y_train, y_test, _preprocessing_name)

            res.append(country_)
            res.append(city_)
            res.append(mae)
            res.append(rmse)
            res.append(r_2)
            res.append(mape)
            res.append('fold_' + str(k_fold))
            res.append(_preprocessing_name)
            res.append('NN')
            print(res)

            full_output = pd.DataFrame({
                'actual_value' : y_test,
                'predict_value' : y_pred,
                'Fold_number' : ['fold_' + str(k_fold)] * len(y_test),
                'date' : date_test
            })
            full_output['city'] = city_
            full_output['country'] = country_
            full_output['pre-processing name'] = _preprocessing_name
            full_output['model_name'] = 'NN'

            summary_value.append(res)
            detail_value.append(full_output)

        k_fold = k_fold + 1
        
        

Amsterdam


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\1398935978.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Netherlands', 'Amsterdam', 1.8209370449270137, 2.4580000442427754, 0.9120477816797461, 7.629899198519972, 'fold_1', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Netherlands', 'Amsterdam', 1.8137539750364815, 2.365208018763634, 0.943925089402315, 7.605094789016851, 'fold_2', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Netherlands', 'Amsterdam', 2.0303707079255955, 2.5618734234031986, 0.9172087443063344, 8.52605276150571, 'fold_3', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Netherlands', 'Amsterdam', 1.87775486136136, 2.3709675911058063, 0.892704521845879, 8.304005059576598, 'fold_4', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Netherlands', 'Amsterdam', 2.4252667492383155, 2.9034864249722974, 0.8921615402419705, 9.517930910390241, 'fold_5', 'StandardScaler', 'NN']
Athens


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\1398935978.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Greece', 'Athens', 2.12461917907576, 2.7707713259002595, 0.9273594279495038, 5.638936172236034, 'fold_1', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Greece', 'Athens', 2.251209522491177, 2.8000049527225874, 0.8996353180309229, 6.831373159780204, 'fold_2', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Greece', 'Athens', 3.1702170567969747, 5.933593742158973, 0.637553919014831, 10.170743679309732, 'fold_3', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Greece', 'Athens', 2.8351752616499115, 3.5672119654390926, 0.8204901744445623, 8.849871085320347, 'fold_4', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Greece', 'Athens', 3.3289201161632804, 3.9667168703366174, 0.838993252926268, 10.49560545950073, 'fold_5', 'StandardScaler', 'NN']
Barcelona


...

10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
['Sewden', 'Stockholm', 1.5126545692331015, 1.9153941844873381, 0.88725565980621, 7.234549456830917, 'fold_1', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Sewden', 'Stockholm', 1.433734095804223, 1.9990277927762037, 0.8911806562215758, 6.181208850249882, 'fold_2', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Sewden', 'Stockholm', 1.8423940340677891, 2.331832902542685, 0.8587811487853584, 8.753734178248722, 'fold_3', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Sewden', 'Stockholm', 1.727821968461825, 2.1202291435222764, 0.7780900687441008, 8.403274233241023, 'fold_4', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Sewden', 'Stockholm', 1.4468844513914902, 1.8067398418719696, 0.8481047475237663, 6.163657273945805, 'fold_5', 'StandardScaler', 'NN']
bern


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\1398935978.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Switzerland', 'bern', 1.6314947976729708, 2.224496580664149, 0.9014179045989346, 6.281009144880508, 'fold_1', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Switzerland', 'bern', 1.526489806501833, 2.0953531982235267, 0.9410232212299288, 6.204218346766527, 'fold_2', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Switzerland', 'bern', 1.62875308511464, 2.1704304266592453, 0.9300395667655719, 6.8304925442304185, 'fold_3', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Switzerland', 'bern', 1.5055642345724585, 1.9623965694516554, 0.9336989345955753, 6.6547317152348, 'fold_4', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Switzerland', 'bern', 3.186982738372943, 3.6037860733648497, 0.6882692737647653, 13.283858799381823, 'fold_5', 'StandardScaler', 'NN']
luxembourg findel airport


C:\Users\HP\AppData\Local\Temp\ipykernel_20712\1398935978.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  city_df.drop(columns=['location','country','latitude','longitude','generationtime_ms','elevation','Date'],inplace=True)


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Luxembourg', 'luxembourg findel airport', 1.8748844470044728, 2.820167856376751, 0.8544323036866841, 7.83756025392216, 'fold_1', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Luxembourg', 'luxembourg findel airport', 1.7870364689935834, 2.2309038582562213, 0.9203085476996201, 7.899419393961357, 'fold_2', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Luxembourg', 'luxembourg findel airport', 1.7973587458536502, 2.251053744714738, 0.9234867381932024, 7.973766531124965, 'fold_3', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Luxembourg', 'luxembourg findel airport', 1.7087787928646558, 2.1276317412996337, 0.9142118055060408, 7.511623493256995, 'fold_4', 'StandardScaler', 'NN']
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step 
['Luxembourg', 'luxembourg findel airport', 3.6049079459551816, 4.022652279027255, 0.7137623420728383, 14.931533260767369, 'fold_5', 'StandardScaler', 'NN']


In [58]:
summary_df = pd.DataFrame(summary_value,columns=['Country','City','MAE','RMSE','R2','MAPE','K_Fold','pre-processing name','model_name'])

In [59]:
detail_df = pd.concat(detail_value)

In [60]:
summary_df.to_csv('../_output/AQI_NN_Summary_result.csv',index=False)

In [61]:
detail_df.to_csv('../_output/AQI_NN_Detail_result.csv',index=False)